In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
# cnn model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot as plt
import copy
import glob
import sklearn
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay,mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import sys
sys.path.append('../utils')

# importing
from emg_data_loading import *

In [7]:
X_vars = [ 'VAR_EMG', 'RMS_EMG', 'IEMG_EMG', 'MAV_EMG', 'LOG_EMG', 'WL_EMG', 'ACC_EMG',
       'DASDV_EMG', 'WAMP_EMG', 'MYOP_EMG', 'FR_EMG', 'MNP_EMG', 'TP_EMG', 'MNF_EMG', 'MDF_EMG',  'WENT_EMG',
           'VAR_MMG', 'RMS_MMG', 'IEMG_MMG', 'MAV_MMG', 'LOG_MMG', 'WL_MMG', 'ACC_MMG',
       'DASDV_MMG', 'WAMP_MMG', 'MYOP_MMG', 'FR_MMG', 'MNP_MMG', 'TP_MMG', 'MNF_MMG', 'MDF_MMG',  'WENT_MMG','muscle_contraction_2', 'muscle_contraction_3', 'prep_wallsit',
       'rest_1', 'rest_2', 'sit_down', 'sit_relaxed_1', 'sit_relaxed_2',
       'wallsit','AM']

outcome = ['cis_subjective_fatigue_lead1']

In [2]:
emg_path ='../../../data/data_features/'
survey_file = '../../../data/survey_data/pre_processed_survey_features.csv'
survey = pd.read_csv(survey_file)
identifiers  = pd.concat([get_dates('Amanda', survey, emg_path)])

In [3]:
indicators_train, indicators_val, indicators_test = identifiers[pd.to_datetime(identifiers['date'])<pd.to_datetime('2024-08-03')],\
    identifiers[(pd.to_datetime(identifiers['date'])<pd.to_datetime('2024-08-07'))*(pd.to_datetime(identifiers['date'])>=pd.to_datetime('2024-08-03'))],\
    identifiers[pd.to_datetime(identifiers['date'])>=pd.to_datetime('2024-08-07')]

In [8]:
data_train = data_loader(indicators_train,survey,emg_path,  outcome=outcome[0])
data_val = data_loader(indicators_val,survey,emg_path,  outcome=outcome[0])
data_test = data_loader(indicators_test,survey,emg_path,  outcome=outcome[0])

In [40]:
# get X and y for both train and test set

data_train = data_train.dropna(subset=X_vars+outcome)
data_test = data_test.dropna(subset=X_vars+outcome)
data_val = data_val.dropna(subset=X_vars+outcome)

X_train, X_val, X_test = np.array(data_train[X_vars]), np.array(data_val[X_vars]), np.array(data_test[X_vars])
y_train, y_val, y_test = np.array(data_train[outcome]).flatten(), np.array(data_val[outcome]).flatten(), np.array(data_test[outcome]).flatten()

In [41]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(X_test)

In [42]:
#accuracy = accuracy_score(y_pred,y_test)
mae= mean_absolute_error(y_pred, y_test)
print("MAE:", mae)
#print("accuracy", accuracy)

MAE: 0.9247805954985931


In [43]:
outcome

['cis_subjective_fatigue_lead1']

In [44]:
out = pd.concat([pd.Series(y_pred).reset_index(),data_loader(indicators_test,survey,emg_path,  outcome=outcome[0]).dropna(subset=X_vars+outcome)[['date']+outcome].reset_index()],axis=1).rename(columns={0:'pred'})
out_daily = out.groupby('date').mean('pred')

In [45]:
out_daily

,index,pred,index,cis_subjective_fatigue_lead1
date,,,,
2024-08-07,14946.5,4.875782,607.500000,2.125
2024-08-08,8384.0,4.851528,606.500000,5.875
2024-08-09,7223.5,4.909302,606.755043,5.375
2024-08-10,13729.5,4.170509,608.500000,6.500
2024-08-12,10203.5,4.722647,605.752577,5.750
2024-08-13,9594.0,5.180410,578.955017,5.250


In [47]:
out_daily.to_csv('/Users/htr365/no_icloud/quantified_self_all/quantified_self/data_processing/ml_pipeline/prediction/amanda_rf_fatigue_daily_prediction.csv')

# "Sanity" Check - Classify interval

In [9]:
#filter_muscle_contractions(data_train)
#filter_relax(data_train)
filter_wallsit(data_train)

,START,END,VAR_EMG,RMS_EMG,IEMG_EMG,MAV_EMG,LOG_EMG,WL_EMG,ACC_EMG,DASDV_EMG,...,prep_wallsit,rest_1,rest_2,sit_down,sit_relaxed_1,sit_relaxed_2,wallsit,cis_subjective_fatigue_lead1,Amanda,AM
788,98500.0,98750.0,0.002722,0.081267,15.577045,0.062308,0.244254,5.996526,0.023986,0.031228,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.000,1,0
789,98625.0,98875.0,0.003857,0.098583,19.140141,0.076561,0.264530,6.837694,0.027351,0.036644,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.000,1,0
790,98750.0,99000.0,0.004422,0.108312,21.373896,0.085496,0.282932,7.016061,0.028064,0.037521,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.000,1,0
791,98875.0,99125.0,0.004012,0.096127,18.077070,0.072308,0.257545,6.442714,0.025771,0.034034,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.000,1,0
792,99000.0,99250.0,0.002989,0.090586,18.057167,0.072229,0.269900,6.760621,0.027042,0.035048,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.000,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,107500.0,107750.0,0.003477,0.100042,20.204559,0.080818,0.278252,7.047876,0.028192,0.035173,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.375,1,0
861,107625.0,107875.0,0.002354,0.083727,17.059057,0.068236,0.262868,6.220413,0.024882,0.031011,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.375,1,0
862,107750.0,108000.0,0.002218,0.075568,14.774455,0.059098,0.242621,5.869968,0.023480,0.029427,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.375,1,0
863,107875.0,108125.0,0.001610,0.062196,11.881109,0.047524,0.218994,5.343569,0.021374,0.027343,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.375,1,0
